In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig,TrainingArguments
from peft import LoraConfig
import torch
from datasets import load_dataset
from trl import SFTTrainer

/data1/debajyoti/test/llms/envllm/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
USE_GPU = True
if USE_GPU:
    device = "cuda:0"
else:
    device = "cpu"

# you may want to change the following parameters depending on your GPU configuration

# free T4 instance
# QUANTIZE_4BIT = True
# USE_GRAD_CHECKPOINTING = True
# TRAIN_BATCH_SIZE = 2
# TRAIN_MAX_SEQ_LENGTH = 512
# USE_FLASH_ATTENTION = False
# GRAD_ACC_STEPS = 16

# equivalent A100 setting
QUANTIZE_4BIT = True
USE_GRAD_CHECKPOINTING = True
TRAIN_BATCH_SIZE = 16
TRAIN_MAX_SEQ_LENGTH = 512
USE_FLASH_ATTENTION = True
GRAD_ACC_STEPS = 2

In [3]:
MODEL_NAME = "CohereForAI/aya-expanse-8b"
models_path = "/data1/debajyoti/test/llms/models/"

attn_implementation = None
if USE_FLASH_ATTENTION:
  attn_implementation="flash_attention_2"

model = AutoModelForCausalLM.from_pretrained(
          MODEL_NAME,
          attn_implementation=attn_implementation,
          torch_dtype=torch.bfloat16,
          cache_dir=models_path,
          device_map="auto"
        )
# model = model.to(device)

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

Loading checkpoint shards: 100%|██████████| 4/4 [00:05<00:00,  1.31s/it]


In [4]:
def get_message_format(prompts):
  messages = []

  for p in prompts:
    messages.append(
          [{"role": "system", "content": """You are a humor recognition assistant judging if an 'Input' is humorous or not. No need to consider any prior context.
            If the 'Input' is humorous, you need to output your final 'Output' as 'Humor'. 
            If the 'Input' is non-humorous, you need to output your final 'Output' as 'Non-humor'."""},
          {"role": "user", "content": p}]
      )

  return messages

def generate_aya(
      model,
      prompts,
      temperature=0.75,
      top_p=1.0,
      top_k=0,
      max_new_tokens=1024
    ):

  # print(prompts)
  messages = get_message_format(prompts)
  # print(messages)

  input_ids = tokenizer.apply_chat_template(
        messages,
        tokenize=True,
        add_generation_prompt=True,
        padding=True,
        return_tensors="pt",
      )
  input_ids = input_ids.to(model.device)
  prompt_padded_len = len(input_ids[0])

  gen_tokens = model.generate(
        input_ids,
        temperature=temperature,
        top_p=top_p,
        top_k=top_k,
        max_new_tokens=max_new_tokens,
        do_sample=True,
      )

  # get only generated tokens
  gen_tokens = [
      gt[prompt_padded_len:] for gt in gen_tokens
    ]

  gen_text = tokenizer.batch_decode(gen_tokens, skip_special_tokens=True)
  return gen_text

In [5]:
# Test generations on langauges in Aya 23 set
prompts = [
    """Input: "Chalo protocol pata chal gaya. Vice President agar Muslim na hote toh shayad kabhi pata hi nahi" 
    Output: """
]

generations = generate_aya(model, prompts)

for p, g in zip(prompts, generations):
  print(
      "PROMPT", p ,"RESPONSE", g, "\n", sep="\n"
    )

PROMPT
Input: "Chalo protocol pata chal gaya. Vice President agar Muslim na hote toh shayad kabhi pata hi nahi" 
    Output: 
RESPONSE
Output: Humor

The statement plays on the idea of a hypothetical situation where the Vice President's actions or decisions might have been different if they were not Muslim, suggesting a subtle critique of religious biases. The use of "chalo" (let's) and the casual tone add to the humorous effect, making it a witty observation.




In [6]:
import pandas as pd
from tqdm import tqdm
import torch
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [7]:
# train_df_eng = pd.read_csv("/data1/debajyoti/code-mix-humor-sarcasm-detection/Dataset/1Humour_English(NEW).csv")
# train_df_eng = train_df_eng[:2000]
# train_df_hin = pd.read_csv("/data1/debajyoti/code-mix-humor-sarcasm-detection/Dataset/1Humour_English(NEW).csv")
# train_df_hin = train_df_hin[:2000]
# train_df = pd.concat([train_df_eng, train_df_hin], ignore_index=True)
# train_df = train_df_hin
train_df = pd.read_csv("/data1/debajyoti/code-mix-humor-sarcasm-detection/Dataset/splits/humor/train.csv")
val_df = pd.read_csv("/data1/debajyoti/code-mix-humor-sarcasm-detection/Dataset/splits/humor/val.csv")
test_df = pd.read_csv("/data1/debajyoti/code-mix-humor-sarcasm-detection/Dataset/splits/humor/test.csv")

test_df

,Sentence,Tag
0,Is 2 takiye ki naukri main mera lakhon ka worl...,0
1,"""Naam : Viv Richards\nBaap ka naam : Master Di...",1
2,Sakshi Maharaj is BJP's Digvijaya Singh.,1
3,Aaj @British_Airways ki toh lag gayi bhai.\nHa...,1
4,Ghanta development hoga! Saare paise to electi...,1
...,...,...
291,"Hum to Aam Aadmi hain ji, woh Megalomaniac hai...",1
292,Neend nahi aati hai raaton mein?,0
293,Ghar mein Bipasha Basu ki photo rakhne se bhoo...,1
294,Aur dikhao aur dikhaopic.twitter.com/Ij4R6xNg9V,0


In [8]:
df = pd.read_csv("/data1/debajyoti/code-mix-humor-sarcasm-detection/outputs/humor/few_shots/16000_hin_eng.csv")
df

,Sentence,Tag
0,दिमाग पैराशूट की तरह हैं।वे खुले होने पर सबसे ...,Humor
1,"राजकुमारी, राजकुमारों के साथ पर्याप्त अनुभव था...",Non-humor
2,"यदि यह पर्यटक का मौसम है, तो मुझे लाइसेंस कहां...",Humor
3,"यदि कोई जॉगर ध्वनि की गति से चलता है, तो क्या ...",Non-humor
4,"जब आपका जीवन एक लाख टुकड़ों में बिखर जाता है, ...",Humor
5,"मेरी प्रेमिका ने सोचा कि यह सिर्फ एक मसौदा है,...",Non-humor
6,आपने मुझे किसी ऐसे व्यक्ति के लिए गलत किया है ...,Humor
7,"बस जब मैं सिरों को पूरा करता हूं, तो कोई छोर ल...",Non-humor


In [9]:
# Convert dataframe to specified format
formatted_few_shot_examples = []
for _, row in df.iterrows():
    formatted_few_shot_examples.append({'role': 'user', 'content': row['Sentence']})
    formatted_few_shot_examples.append({'role': 'assistant', 'content': row['Tag']})
    
formatted_few_shot_examples

[{'role': 'user',
  'content': 'दिमाग पैराशूट की तरह हैं।वे खुले होने पर सबसे अच्छा काम करते हैं।बस सुनिश्चित करें कि तार अभी भी संलग्न हैं।'},
 {'role': 'assistant', 'content': 'Humor'},
 {'role': 'user',
  'content': 'राजकुमारी, राजकुमारों के साथ पर्याप्त अनुभव था, मेंढक की तलाश करता है।'},
 {'role': 'assistant', 'content': 'Non-humor'},
 {'role': 'user',
  'content': 'यदि यह पर्यटक का मौसम है, तो मुझे लाइसेंस कहां मिलेगा?'},
 {'role': 'assistant', 'content': 'Humor'},
 {'role': 'user',
  'content': 'यदि कोई जॉगर ध्वनि की गति से चलता है, तो क्या वह अभी भी अपने वॉकमैन को सुन सकता है?'},
 {'role': 'assistant', 'content': 'Non-humor'},
 {'role': 'user',
  'content': 'जब आपका जीवन एक लाख टुकड़ों में बिखर जाता है, तो टुकड़ों को उठाएं, कुछ गोंद को पकड़ें, और एक नया बनाएं।'},
 {'role': 'assistant', 'content': 'Humor'},
 {'role': 'user',
  'content': 'मेरी प्रेमिका ने सोचा कि यह सिर्फ एक मसौदा है, इसलिए उसने किनारों को स्कैलप किया और इसे कपकेक लाइनर के लिए इस्तेमाल किया।'},
 {'role': 'assist

In [10]:
formatted_few_shot_examples = ""
for index, row in df.iterrows():
    # if index == 3: # or index == 4:
    #     formatted_few_shot_examples += f"""'Input': "{row['Sentence']}"\n'Response':"{row['Tag']}"\n\n"""
    formatted_few_shot_examples += f"""'Input': "{row['Sentence']}"\n'Response':"{row['Tag']}"\n\n"""
    # if index == 1:
    #     break

# Display the result
print(formatted_few_shot_examples)

'Input': "दिमाग पैराशूट की तरह हैं।वे खुले होने पर सबसे अच्छा काम करते हैं।बस सुनिश्चित करें कि तार अभी भी संलग्न हैं।"
'Response':"Humor"

'Input': "राजकुमारी, राजकुमारों के साथ पर्याप्त अनुभव था, मेंढक की तलाश करता है।"
'Response':"Non-humor"

'Input': "यदि यह पर्यटक का मौसम है, तो मुझे लाइसेंस कहां मिलेगा?"
'Response':"Humor"

'Input': "यदि कोई जॉगर ध्वनि की गति से चलता है, तो क्या वह अभी भी अपने वॉकमैन को सुन सकता है?"
'Response':"Non-humor"

'Input': "जब आपका जीवन एक लाख टुकड़ों में बिखर जाता है, तो टुकड़ों को उठाएं, कुछ गोंद को पकड़ें, और एक नया बनाएं।"
'Response':"Humor"

'Input': "मेरी प्रेमिका ने सोचा कि यह सिर्फ एक मसौदा है, इसलिए उसने किनारों को स्कैलप किया और इसे कपकेक लाइनर के लिए इस्तेमाल किया।"
'Response':"Non-humor"

'Input': "आपने मुझे किसी ऐसे व्यक्ति के लिए गलत किया है जो लानत देता है।"
'Response':"Humor"

'Input': "बस जब मैं सिरों को पूरा करता हूं, तो कोई छोर ले जाता है।"
'Response':"Non-humor"




### Create prompt, parse response and  generate response

In [11]:
from transformers import pipeline
import re
import json

# Define the function to create the prompt
def create_prompt(input, num_examples):

    # System prompt 
    # system_prompt = """You are a humor recognition assistant judging if an 'Input' is humor or not. 
    #     If the 'Input' is humorous, you need to give your final 'Output' as "Humor". 
    #     If the 'Input' is non-humorous, you need to give your final 'Output' as "Non-humor". Dont output anything else. """

    # messages = [system_prompt + formatted_few_shot_examples, f"\'Input\':'{input}'\n\'Response\':"]
    messages = f"""
                {formatted_few_shot_examples}
                \'Input\':'{input}'
                \'Response\':
    """
    # messages = [system_prompt, input]

    return messages

def parse_response(response):
    """
    Extracts the classification label and explanation from the model-generated response,
    starting from the line containing the final "Answer:".

    Args:
        response (str): The response generated by the model.

    Returns:
        tuple: A tuple containing the classification label ("Humor" or "Non-humor") 
               and the explanation text.
    """
    # print(response)
    if len(response) > 0:        
        if "Non-humor" in response:
            return "0"
        elif "Humor" in response:
            return "1"
        else:# If no label is found, return None
            return None


In [12]:
def process_data(output_filepath):
    # df = pd.read_csv(data_filepath)
    df = test_df
    # print(df)
    # df = df[:5]
    
    
    results = []
    num_examples = 15 # number of shots in few-shot
    # Loop over the DataFrame and pass the text to generate response
    for index, row in tqdm(list(df.iterrows())):
        # print(row)
        prompt = create_prompt(row['Sentence'], num_examples)
        # print(prompt)
        generations = generate_aya(model, [prompt])
        # print(generations[0])
        generated_label = parse_response(generations[0])
        # print(generated_label)

        # # veracity, explanation = parse_response(model_response)
        
        results.append({
            'Sentence': row['Sentence'],
            'Label': row['Tag'],
            'Response': generations[0],
            'Gen_label': generated_label,
            # 'Explanation': explanation
        })
        # break
    
    results = pd.DataFrame(results)
    
    return results

# Example file paths, replace with your actual paths
output_filepath = '/data1/debajyoti/test/llms/data/humor'

# Run the data processing
result = process_data(output_filepath)

100%|██████████| 296/296 [21:28<00:00,  4.35s/it]


In [13]:
result

,Sentence,Label,Response,Gen_label
0,Is 2 takiye ki naukri main mera lakhon ka worl...,0,'Humor'\n\nयह वाक्य एक हास्यप्रद तुलना प्रस्तु...,1
1,"""Naam : Viv Richards\nBaap ka naam : Master Di...",1,"'Response': ""Non-humor""\n\nयह इनपुट एक व्यक्ति...",0
2,Sakshi Maharaj is BJP's Digvijaya Singh.,1,'Non-humor'\n\nदिए गए इनपुट में कोई हास्यास्पद...,0
3,Aaj @British_Airways ki toh lag gayi bhai.\nHa...,1,'Humor'\n\nइस इनपुट में एक हल्के-फुल्के और मजा...,1
4,Ghanta development hoga! Saare paise to electi...,1,'Humor'\n\nयह वाक्यांश व्यंग्यात्मक और हास्यप्...,1
...,...,...,...,...
291,"Hum to Aam Aadmi hain ji, woh Megalomaniac hai...",1,'Humor'\n\nयह इनपुट में एक व्यंग्यात्मक और हास...,1
292,Neend nahi aati hai raaton mein?,0,'Humor'\n\nयह वाक्यांश एक आम समस्या को हास्यपू...,1
293,Ghar mein Bipasha Basu ki photo rakhne se bhoo...,1,'Humor'\n\nयह वाक्य एक हास्यप्रद छवि पेश करता ...,1
294,Aur dikhao aur dikhaopic.twitter.com/Ij4R6xNg9V,0,'Non-humor'\n\nइस इनपुट में कोई हास्यप्रद तत्व...,0
